In [205]:
#read in the data
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import math



gender_meta = pd.read_csv('df_gender.csv')
df_meta = pd.read_csv('../metadata/19cfictionmeta.tsv', sep='\t')
meta = pd.read_csv('../metadata/enriched_metadata.tsv', sep='\t')

df_meta = pd.read_csv('df_gender.csv')

In [206]:
meta.head()

,BOOK_ID,FILENAME,LIBRARIES,TITLE,AUTH_LAST,AUTH_FIRST,AUTH_ID,WRITTEN_AS,PUBL_CITY,PUBLISHER,...,birthyear,author_age,is_bestseller,bestseller_contrast,is_reviewed,reviewed_contrast,is_syllabus,syllabus_contrast,is_discussed,discussed_contrast
0,10517,00010517.txt,49.0,Place in the city,Fast,Howard,A_01139,NaN,New York,"""Harcourt, Brace & Co.""",...,1919.0,18.0,0,0,0,0,0,0,0,0
1,uc1.$b243800,uc1.$b243800.txt,NaN,The trap : a novel,Jacobson,['Dan'],NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0
2,21599,00021599.txt,331.0,Umbertina,Barolini,Helen,A_00173,NaN,New York,Seaview Books,...,1925.0,54.0,0,0,0,0,0,0,1,0
3,uc1.b4372570,uc1.b4372570.txt,NaN,The Odyssey of a nice girl,Suckow,['Ruth'],NaN,NaN,NaN,NaN,...,1892.0,33.0,0,0,0,0,0,0,0,0
4,loc.ark+=13960=t98636g92,loc.ark+=13960=t98636g92.txt,NaN,Granville;,Guttersen,['Granville'],NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


In [207]:
df_all = []

for decade in range(10, 80, 5):
    data = pd.read_csv(f'../precocity/precocity_tuned_19{decade}s_docs.tsv', sep='\t')
    data['docid'] = data['docid'].astype(str)
    data.set_index('docid', inplace=True)
    df_all.append(data)

# If you want to concatenate into a single DataFrame:
df_all = pd.concat(df_all, axis=0)


In [208]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience
docid,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269


In [209]:
meta['author'] = meta['AUTH_FIRST'] + ' ' + meta['AUTH_LAST']

In [210]:
meta['docid'] = meta['BOOK_ID']

In [211]:
meta['docid']

0                          10517
1                   uc1.$b243800
2                          21599
3                   uc1.b4372570
4       loc.ark+=13960=t98636g92
                  ...           
8913                       10629
8914                uc1.b3710792
8915                       20619
8916    uc2.ark+=13960=t0sq8qh13
8917    uc2.ark+=13960=t26974971
Name: docid, Length: 8918, dtype: object

In [212]:
# df_all['docid']

In [213]:
df_all.columns


Index(['date', 'num_chunks', 'fraction_compared', 'filtered', 'time_radius',
       'chunks_used', 'precocity', 'novelty', 'transience'],
      dtype='object')

In [214]:
author_dict = pd.Series(meta['author'].values, index=meta['docid']).to_dict()
df_all['author'] = df_all.index.map(author_dict).fillna('unknown')

In [215]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author
docid,,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269,Frederick Landis
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858,Frederick Landis
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364,Frederick Landis
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578,Frederick Landis
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269,Frederick Landis


In [216]:
len(df_meta)

18428

In [217]:
# df_all = pd.read_csv('df_all_fiction.csv', dtype = str)


In [218]:
# gender_dict = pd.Series(df_meta['likelygender'].values, index=df_meta['author']).to_dict()
# df_all['likelygender'] = df_all['author'].map(gender_dict).fillna('unknown')
# 


In [219]:
discussed = meta.loc[meta.is_discussed == True].index.tolist()
# dis_kld_precoc.extend(df_all.loc[df_all.index.isin(discussed), 'precocity'])


In [220]:
# df_all = pd.read_csv('df_all_fiction.csv', dtype = 'str')

In [221]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author
docid,,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269,Frederick Landis
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858,Frederick Landis
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364,Frederick Landis
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578,Frederick Landis
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269,Frederick Landis


In [222]:
def flip_name(n):
    if ',' in n:
        last, first = n.split(',', 1)
        return f"{first.strip()} {last.strip()}"
    return n  # leave unchanged

df_meta['author'] = df_meta['author'].astype(str).apply(flip_name)

In [223]:
df_meta.head()

,Unnamed: 0,author,gender,likelygender
0,0,William Edmondstoune Aytoun,m,m
1,1,Arthur M Winfield,m,m
2,2,nan,NaN,NaN
3,3,Ivan Doig,m,m
4,4,Amy Tan,f,f


In [224]:
def clean_author(author):
    words = []
    for word in author.lower().split(' '):
        word.strip()
        word.lower()
        words.append(word)
    author = ' '.join(words)
    return author

In [225]:
df_all['clean_author'] = df_all['author'].apply(clean_author)

In [226]:
df_meta['clean_author'] = df_meta['author'].apply(clean_author)


In [227]:
df_meta.head()

,Unnamed: 0,author,gender,likelygender,clean_author
0,0,William Edmondstoune Aytoun,m,m,william edmondstoune aytoun
1,1,Arthur M Winfield,m,m,arthur m winfield
2,2,nan,NaN,NaN,nan
3,3,Ivan Doig,m,m,ivan doig
4,4,Amy Tan,f,f,amy tan


In [228]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author
docid,,,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858,Frederick Landis,frederick landis
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364,Frederick Landis,frederick landis
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578,Frederick Landis,frederick landis
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis


In [229]:
gender_dict = pd.Series(df_meta['likelygender'].values, index=df_meta['clean_author']).to_dict()
df_all['likelygender'] = df_all['clean_author'].map(gender_dict).fillna('unknown')


In [230]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author,likelygender
docid,,,,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis,unknown


In [231]:
df_all.loc[df_all['likelygender'] != 'unknown']

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author,likelygender
docid,,,,,,,,,,,,
3269,1910,135,1.00,no filter,10,1.00,0.056471,4.826024,4.769552,Jack London,jack london,m
3269,1910,135,1.00,no filter,10,0.25,0.135420,4.902755,4.767334,Jack London,jack london,m
3269,1910,135,1.00,no filter,20,1.00,-0.009875,4.867934,4.877810,Jack London,jack london,m
3269,1910,135,1.00,no filter,20,0.25,0.028385,4.940307,4.911922,Jack London,jack london,m
3269,1910,135,1.00,train,10,1.00,0.056471,4.826024,4.769552,Jack London,jack london,m
...,...,...,...,...,...,...,...,...,...,...,...,...
21646,1979,144,0.05,trainauth,20,0.25,0.074612,3.664597,3.589985,Charles McCarry,charles mccarry,m
21646,1979,144,0.05,trainauthquote,10,1.00,-0.006940,3.551042,3.557983,Charles McCarry,charles mccarry,m
21646,1979,144,0.05,trainauthquote,10,0.25,0.032853,3.597493,3.564640,Charles McCarry,charles mccarry,m


In [232]:
meta.head()

,BOOK_ID,FILENAME,LIBRARIES,TITLE,AUTH_LAST,AUTH_FIRST,AUTH_ID,WRITTEN_AS,PUBL_CITY,PUBLISHER,...,is_bestseller,bestseller_contrast,is_reviewed,reviewed_contrast,is_syllabus,syllabus_contrast,is_discussed,discussed_contrast,author,docid
0,10517,00010517.txt,49.0,Place in the city,Fast,Howard,A_01139,NaN,New York,"""Harcourt, Brace & Co.""",...,0,0,0,0,0,0,0,0,Howard Fast,10517
1,uc1.$b243800,uc1.$b243800.txt,NaN,The trap : a novel,Jacobson,['Dan'],NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,['Dan'] Jacobson,uc1.$b243800
2,21599,00021599.txt,331.0,Umbertina,Barolini,Helen,A_00173,NaN,New York,Seaview Books,...,0,0,0,0,0,0,1,0,Helen Barolini,21599
3,uc1.b4372570,uc1.b4372570.txt,NaN,The Odyssey of a nice girl,Suckow,['Ruth'],NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,['Ruth'] Suckow,uc1.b4372570
4,loc.ark+=13960=t98636g92,loc.ark+=13960=t98636g92.txt,NaN,Granville;,Guttersen,['Granville'],NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,['Granville'] Guttersen,loc.ark+=13960=t98636g92


In [233]:

# We also create a list of the docids that have been discussed
# in critical articles, and a contrast set that have the 
# same distribution over time but are not mentioned in our
# literary studies corpus.

discussed = meta.loc[meta.is_discussed == True].docid.tolist()
# discussed_contrast = meta.loc[meta.discussed_contrast == True].index.tolist()
# 
# print('We have ', len(discussed), ' discussed docs and ', len(discussed_contrast), ' contrast docs.')

In [234]:
discussed

['21599',
 '20447',
 '10911',
 '20321',
 '10370',
 '4599',
 '21697',
 '10283',
 '11352',
 '20832',
 '20744',
 '10591',
 '4079',
 '21642',
 '21358',
 '20877',
 '21187',
 '11111',
 '11308',
 '20040',
 '11096',
 '10522',
 '21180',
 '20125',
 '23522',
 '4665',
 '20681',
 '20497',
 '11356',
 '10462',
 '20023',
 '11497',
 '21374',
 '21496',
 '11043',
 '10418',
 '21040',
 '21072',
 '20170',
 '21451',
 '10224',
 '20986',
 '11664',
 '20592',
 '10961',
 '10783',
 '11056',
 '10483',
 '21085',
 '11446',
 '10347',
 '10375',
 '21130',
 '4868',
 '20861',
 '20188',
 '20721',
 '20283',
 '21668',
 '20586',
 '3377',
 '10732',
 '11182',
 '10691',
 '11777',
 '23571',
 '10221',
 '11124',
 '10379',
 '10476',
 '23556',
 '10461',
 '11060',
 '10794',
 '20500',
 '10273',
 '21456',
 '11084',
 '10683',
 '20422',
 '11031',
 '20436',
 '20088',
 '11016',
 '21044',
 '21199',
 '20564',
 '20745',
 '10405',
 '10916',
 '10795',
 '11026',
 '20754',
 '21231',
 '4842',
 '20897',
 '10258',
 '11007',
 '10887',
 '20245',
 '2011

In [235]:

def print_effect_size(t, p, df):
    ''' Calculates Cohen's d and r2
    for t-test statistics.'''

    d = 2*t / math.sqrt(df)
    r2 = t**2 / (t**2 + df)

    print(f'Cohens d is {d} and r2 is {r2}.')

In [236]:
# dis_kld_precoc = []
# # notdis_kld_precoc = []
# 
# for decade in range(10, 80, 5):
# 	
# 	data = pd.read_csv('../precocity/precocity_tuned_19' + str(decade) + 's_docs.tsv', sep = '\t')
# 	data['docid'] = data['docid'].astype(str)
# 	data.set_index('docid', inplace=True)
#     
# 
# 	selected = pd.DataFrame(data.loc[(data.time_radius == 20) & (data.filtered == 'trainauth') & (data.chunks_used == .25) & (data.fraction_compared == 1.0), :])
# 	# dis_kld_precoc.extend(selected.loc[selected.index.isin(discussed), 'precocity'])
# 	# notdis_kld_precoc.extend(selected.loc[selected.index.isin(discussed_contrast), 'precocity'])
# 	dis_kld_precoc.extend(selected.loc[selected.index.isin(discussed), 'precocity'])
#     
# 
# # t, p = ttest_ind(dis_kld_precoc, notdis_kld_precoc)
# # df = len(dis_kld_precoc) + len(notdis_kld_precoc) - 2
# # print(t, p, df)
# # print_effect_size(t, p, df)

In [237]:
selected

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,likelygender,clean_author,is_discussed
docid,,,,,,,,,,,,,
3262,1910,7,1.0,trainauth,20,0.25,0.030063,5.273641,5.243578,Frederick Landis,unknown,frederick landis,False
pst.000030852204,1910,6,1.0,trainauth,20,0.25,-0.011982,6.718890,6.730873,['Mark'] Twain,unknown,['mark'] twain,False
uva.x030743227,1910,45,1.0,trainauth,20,0.25,-0.041834,5.086374,5.128208,['Evelyn'] Raymond,unknown,['evelyn'] raymond,False
3287,1910,94,1.0,trainauth,20,0.25,-0.038774,4.744162,4.782936,Meredith Nicholson,unknown,meredith nicholson,False
3269,1910,135,1.0,trainauth,20,0.25,0.027396,4.940820,4.913424,Jack London,m,jack london,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21666,1979,29,1.0,trainauth,20,0.25,0.130510,5.120206,4.989696,Spider Robinson,unknown,spider robinson,False
21656,1979,163,1.0,trainauth,20,0.25,0.132888,4.904528,4.771640,G. J. A. O'Toole,unknown,g. j. a. o'toole,False
21607,1979,96,1.0,trainauth,20,0.25,0.140529,5.018054,4.877525,C. Terry Cline,unknown,c. terry cline,False


In [238]:
selected.loc[selected.index.isin(discussed)]

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,likelygender,clean_author,is_discussed
docid,,,,,,,,,,,,,
4842,1910,193,1.0,trainauth,20,0.25,0.067649,5.012466,4.944818,H. G. Wells,unknown,h. g. wells,True
4840,1910,197,1.0,trainauth,20,0.25,0.089616,4.617200,4.527584,Arnold Bennett,m,arnold bennett,True
3285,1910,73,1.0,trainauth,20,0.25,0.028102,5.757080,5.728978,Clarence Edward Mulford,m,clarence edward mulford,True
3248,1910,76,1.0,trainauth,20,0.25,0.024521,5.047486,5.022964,O. Henry,unknown,o. henry,True
3255,1910,68,1.0,trainauth,20,0.25,0.044970,4.999555,4.954585,Henry James,m,henry james,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,1979,53,1.0,trainauth,20,0.25,0.120583,4.602524,4.481941,Philip Roth,m,philip roth,True
23571,1979,156,1.0,trainauth,20,0.25,0.097123,4.791334,4.694211,Nadine Gordimer,f,nadine gordimer,True
21616,1979,162,1.0,trainauth,20,0.25,0.092633,4.845042,4.752409,Howard Fast,m,howard fast,True


In [239]:
# dis_kld_precoc.extend(selected.loc[selected.index.isin(discussed), 'precocity'])


In [240]:
dis_kld_precoc

[]

In [241]:
# data = df_all
# 
# selected = pd.DataFrame(data.loc[(data.time_radius == 20) & (data.filtered == 'trainauth') & (data.chunks_used == .25) & (data.fraction_compared == 1.0), :])
# 
# 
# dis_kld_precoc.extend(selected.loc[selected.index.isin(discussed), 'precocity'])


In [242]:
# selected

In [243]:
dis_kld_precoc

[]

In [244]:
# import statsmodels.formula.api as smf
# 
# model = smf.logit("is_discussed ~ C(likelygender) + precocity + C(likelygender):precocity + decade", data=dis_kld_precoc.fit())
# print(model.summary())



In [245]:
# df_all['gender'].value_counts(dropna=False)


KeyError: 'gender'

In [246]:
df_all.head()

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author,likelygender
docid,,,,,,,,,,,,
3262,1910,7,1.0,no filter,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,10,0.25,0.094322,4.838180,4.743858,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,20,1.00,-0.030010,5.178354,5.208364,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,no filter,20,0.25,0.030063,5.273641,5.243578,Frederick Landis,frederick landis,unknown
3262,1910,7,1.0,train,10,1.00,0.051122,5.131392,5.080269,Frederick Landis,frederick landis,unknown


In [247]:
data = df_all
selected = pd.DataFrame(data.loc[(data.time_radius == 20) & (data.filtered == 'trainauth') & (data.chunks_used == .25) & (data.fraction_compared == 1.0), :])

In [248]:
discussed

['21599',
 '20447',
 '10911',
 '20321',
 '10370',
 '4599',
 '21697',
 '10283',
 '11352',
 '20832',
 '20744',
 '10591',
 '4079',
 '21642',
 '21358',
 '20877',
 '21187',
 '11111',
 '11308',
 '20040',
 '11096',
 '10522',
 '21180',
 '20125',
 '23522',
 '4665',
 '20681',
 '20497',
 '11356',
 '10462',
 '20023',
 '11497',
 '21374',
 '21496',
 '11043',
 '10418',
 '21040',
 '21072',
 '20170',
 '21451',
 '10224',
 '20986',
 '11664',
 '20592',
 '10961',
 '10783',
 '11056',
 '10483',
 '21085',
 '11446',
 '10347',
 '10375',
 '21130',
 '4868',
 '20861',
 '20188',
 '20721',
 '20283',
 '21668',
 '20586',
 '3377',
 '10732',
 '11182',
 '10691',
 '11777',
 '23571',
 '10221',
 '11124',
 '10379',
 '10476',
 '23556',
 '10461',
 '11060',
 '10794',
 '20500',
 '10273',
 '21456',
 '11084',
 '10683',
 '20422',
 '11031',
 '20436',
 '20088',
 '11016',
 '21044',
 '21199',
 '20564',
 '20745',
 '10405',
 '10916',
 '10795',
 '11026',
 '20754',
 '21231',
 '4842',
 '20897',
 '10258',
 '11007',
 '10887',
 '20245',
 '2011

In [249]:
selected

,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author,likelygender
docid,,,,,,,,,,,,
3262,1910,7,1.0,trainauth,20,0.25,0.030063,5.273641,5.243578,Frederick Landis,frederick landis,unknown
pst.000030852204,1910,6,1.0,trainauth,20,0.25,-0.011982,6.718890,6.730873,['Mark'] Twain,['mark'] twain,unknown
uva.x030743227,1910,45,1.0,trainauth,20,0.25,-0.041834,5.086374,5.128208,['Evelyn'] Raymond,['evelyn'] raymond,unknown
3287,1910,94,1.0,trainauth,20,0.25,-0.038774,4.744162,4.782936,Meredith Nicholson,meredith nicholson,unknown
3269,1910,135,1.0,trainauth,20,0.25,0.027396,4.940820,4.913424,Jack London,jack london,m
...,...,...,...,...,...,...,...,...,...,...,...,...
21666,1979,29,1.0,trainauth,20,0.25,0.130510,5.120206,4.989696,Spider Robinson,spider robinson,unknown
21656,1979,163,1.0,trainauth,20,0.25,0.132888,4.904528,4.771640,G. J. A. O'Toole,g. j. a. o'toole,unknown
21607,1979,96,1.0,trainauth,20,0.25,0.140529,5.018054,4.877525,C. Terry Cline,c. terry cline,unknown


In [250]:
selected['is_discussed'] = False


In [251]:
selected.loc[selected.index.isin(discussed), 'is_discussed'] = True


In [252]:
selected.loc[selected.index.isin(discussed)]


,date,num_chunks,fraction_compared,filtered,time_radius,chunks_used,precocity,novelty,transience,author,clean_author,likelygender,is_discussed
docid,,,,,,,,,,,,,
4842,1910,193,1.0,trainauth,20,0.25,0.067649,5.012466,4.944818,H. G. Wells,h. g. wells,unknown,True
4840,1910,197,1.0,trainauth,20,0.25,0.089616,4.617200,4.527584,Arnold Bennett,arnold bennett,m,True
3285,1910,73,1.0,trainauth,20,0.25,0.028102,5.757080,5.728978,Clarence Edward Mulford,clarence edward mulford,m,True
3248,1910,76,1.0,trainauth,20,0.25,0.024521,5.047486,5.022964,O. Henry,o. henry,unknown,True
3255,1910,68,1.0,trainauth,20,0.25,0.044970,4.999555,4.954585,Henry James,henry james,m,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21668,1979,53,1.0,trainauth,20,0.25,0.120583,4.602524,4.481941,Philip Roth,philip roth,m,True
23571,1979,156,1.0,trainauth,20,0.25,0.097123,4.791334,4.694211,Nadine Gordimer,nadine gordimer,f,True
21616,1979,162,1.0,trainauth,20,0.25,0.092633,4.845042,4.752409,Howard Fast,howard fast,m,True


In [253]:
df_filtered = selected.loc[selected['likelygender'].isin(['m','f'])]

In [254]:
df_filtered = df_filtered[['likelygender','precocity', 'is_discussed', 'date']]

In [255]:
df_filtered.head()

,likelygender,precocity,is_discussed,date
docid,,,,
3269,m,0.027396,False,1910
3215,m,0.030150,False,1910
3251,m,-0.034197,False,1910
3210,m,0.045705,False,1910
3270,m,0.030594,False,1910


In [262]:
df_filtered = pd.get_dummies(df_filtered, columns=['likelygender'])


In [264]:
# df = pd.get_dummies(df_filtered, columns=['likelygender'])
df_filtered['is_discussed'] = df_filtered['is_discussed'].astype(int)
df_filtered['likelygender_m'] = df_filtered['likelygender_m'].astype(int)


In [265]:
df_filtered.drop(['likelygender_f'], axis=1, inplace=True)

In [268]:
import statsmodels.formula.api as smf

model = smf.logit("is_discussed ~ C(likelygender_m) + precocity + C(likelygender_m):precocity + date", data=df_filtered)
# print(model.summary())
result = model.fit()       # fit the model
print(result.summary())  

Optimization terminated successfully.
         Current function value: 0.399742
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           is_discussed   No. Observations:                 2289
Model:                          Logit   Df Residuals:                     2284
Method:                           MLE   Df Model:                            4
Date:                Sun, 07 Dec 2025   Pseudo R-squ.:                 0.01777
Time:                        14:48:30   Log-Likelihood:                -915.01
converged:                       True   LL-Null:                       -931.56
Covariance Type:            nonrobust   LLR p-value:                 1.139e-06
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -0.8577      6.494     -0.13